# Amazon Bedrock Knowledge Bases - Amazon Bedrock Data Automation을 활용한 오디오·비디오 데이터 준비

# 소개

이 모듈은 Amazon Bedrock Data Automation(BDA)과 Bedrock Knowledge Bases(KB)를 사용해 멀티모달 Retrieval-Augmented Generation(RAG) 애플리케이션을 구축하는 방법을 설명합니다. 애플리케이션은 비디오와 오디오를 포함한 멀티모달 데이터를 분석하고 인사이트를 생성하도록 설계되었습니다. 자체 데이터 소스의 컨텍스트 정보를 BDA와 결합하면 정확하고 안전한 지능형 검색 생성형 AI 애플리케이션을 만들 수 있습니다.

이 노트북은 지능형 검색 애플리케이션을 구축하는 첫 번째 단계로, BDA를 사용해 비디오와 오디오 데이터를 효율적으로 처리하고 KB 임베딩을 위한 컨텍스트 출력물을 생성하는 방법을 보여 줍니다.

최신 BDA와 Amazon Bedrock Knowledge Bases의 통합으로, Bedrock Knowledge Bases 데이터 소스의 파서로 BDA를 지정할 수 있습니다.

## 주요 기능

- Amazon Bedrock Data Automation(BDA): 멀티모달 데이터에서 콘텐츠를 자동으로 추출하는 관리형 서비스입니다. BDA는 문서, 이미지, 오디오, 비디오 등 비정형 멀티모달 콘텐츠에서 유용한 인사이트를 생성하는 통합 멀티모달 추론 API를 제공합니다.

- BDA와 함께 RAG 솔루션을 구축하는 Bedrock KB: Amazon Bedrock KB는 BDA를 사용해 멀티모달 콘텐츠를 추출하고, 선택한 임베딩 모델로 의미 임베딩을 생성해 지정된 벡터 스토어에 저장합니다. 이를 통해 텍스트뿐 아니라 이미지, 비디오, 오디오 데이터에 기반한 질문에도 응답을 검색하고 생성할 수 있습니다. 또한 검색 결과에는 시각적 데이터에 대한 출처 정보가 포함되어 투명성과 신뢰도를 높여 줍니다.

## 사전 요구 사항
Amazon Bedrock 콘솔에서 `Anthropic Claude 3 Sonnet`, `Amazon Nova Micro`, `Titan Text Embeddings V2` 모델에 대한 접근 권한을 활성화하세요.

이 노트북을 실행하려면 적절한 IAM 역할 권한이 필요합니다. 계정에 있는 기존 IAM 역할을 사용하거나 새 역할을 생성할 수 있으며, 해당 역할에는 BDA, Bedrock KB, IAM 역할 생성, SageMaker, S3 API를 호출할 수 있는 권한이 있어야 합니다.

참고: 조직의 보안 정책이 허용한다면 AdministratorAccess IAM 정책을 사용할 수 있습니다.

<div class="alert alert-block alert-info">
<b>참고:</b> "Run All Cells" 옵션 대신 노트북 셀을 하나씩 실행하세요.
</div>

# 노트북 및 boto3 클라이언트 설정

이 단계에서는 노트북 전반에서 사용할 필수 라이브러리를 임포트합니다. Amazon Bedrock Data Automation(BDA)을 boto3와 함께 사용하려면 최신 AWS SDK for Python(boto3) 버전이 필요합니다. Boto3 1.35.96 이상 버전이 요구됩니다.

참고: 퍼블릭 프리뷰 출시 시점에는 BDA가 us-west-2 리전에서만 사용 가능합니다.

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

<div class="alert alert-block alert-info">
<b>참고:</b> 이 워크숍에서는 "kb-bda-multimodal-datasource-{account_id}" 명명 규칙을 따르는 새 S3 버킷을 사용하며, 입력과 출력은 기본 버킷의 "bda" 폴더에 저장됩니다.
</div>

In [ ]:
import boto3
from botocore.exceptions import ClientError
import json, uuid
from datetime import datetime
import time
from time import sleep
import random
from tqdm import tqdm
from pathlib import Path
from IPython.display import JSON, IFrame, Audio, display, clear_output
import IPython.display as display
import logging
import sys
from pathlib import Path

# Get current path and go up two parent directories
current_path = Path().resolve()
parent_path = current_path.parent.parent  # Go up two levels

# Add to sys.path if not already there
if str(parent_path) not in sys.path:
    sys.path.append(str(parent_path))

# Now you can import from utils
from utils.knowledge_base import BedrockKnowledgeBase


In [ ]:
!pip freeze | grep boto3

In [ ]:
#Clients
suffix = random.randrange(200, 900)

sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

bucket_name_kb = f'bedrock-kb-{suffix}-1' # replace it with your first bucket name.
region_name = "us-west-2" # can be removed ones BDA is GA and available in other regions.
region = region_name

s3_client = boto3.client('s3', region_name=region_name)

bda_client = boto3.client('bedrock-data-automation', region_name=region_name)
bda_runtime_client = boto3.client('bedrock-data-automation-runtime', region_name=region_name)

bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Function to check if the bucket exists, if not, create the data_bucket
from utils.knowledge_base_operators import bucket_exists
            
# Create S3 bucket for the KB if it doesn't exist
if not bucket_exists(bucket_name_kb):
    print(f"Bucket '{bucket_name_kb}' does not exist. Creating it now...")
    if region == "us-east-1":
        s3_client.create_bucket(Bucket=bucket_name_kb)
    else:
        s3_client.create_bucket(
            Bucket=bucket_name_kb,
            CreateBucketConfiguration={'LocationConstraint': region}
        )
    print(f"Bucket '{bucket_name_kb}' created successfully.")
else:
    print(f"Bucket '{bucket_name_kb}' already exists.")


bucket_name_input = f's3://{bucket_name_kb}/bda/input'      # DBA input path
bucket_name_output = f's3://{bucket_name_kb}/bda/output'    # DBA output path

## BDA 프로젝트 생성
BDA 작업을 시작하려면 표준 및 사용자 지정 출력 구성을 관리하는 BDA 프로젝트가 필요합니다. 프로젝트는 재사용 가능하므로 동일한 설정을 공유하는 여러 비디오/오디오 파일을 처리할 때 동일한 구성을 적용할 수 있습니다.

In [ ]:
project_name= f'bda-workshop-kb-project-{str(uuid.uuid4())[0:4]}'

# delete project if it already exists
projects_existing = [project for project in bda_client.list_data_automation_projects()["projects"] if project["projectName"] == project_name]
if len(projects_existing) >0:
    print(f"Deleting existing project: {projects_existing[0]}")
    bda_client.delete_data_automation_project(projectArn=projects_existing[0]["projectArn"])

In [ ]:
#BDA API standard output schema
response = bda_client.create_data_automation_project(
    projectName=project_name,
    projectDescription='BDA video processing project',
    projectStage='DEVELOPMENT',
    standardOutputConfiguration={
        "video": {
            "extraction": {
                "category": {
                    "state": "ENABLED",
                    "types": ["CONTENT_MODERATION", "TEXT_DETECTION", "TRANSCRIPT"]
                },
                "boundingBox": {"state": "ENABLED"}
            },
            "generativeField": {
                "state": "ENABLED",
                "types": ["VIDEO_SUMMARY", "CHAPTER_SUMMARY", "IAB"]
            }
        },
        "audio": {
            "extraction": {
                "category": {
                    "state": "ENABLED", 
                    "types": ["AUDIO_CONTENT_MODERATION", "TOPIC_CONTENT_MODERATION", "TRANSCRIPT"]
                }
            },
            "generativeField": {
                "state": "ENABLED",
                "types": ["AUDIO_SUMMARY", "TOPIC_SUMMARY", "IAB"]
            }
        }
    }
)

In [ ]:
kb_project_arn = response.get("projectArn")
print("BDA kb project ARN:", kb_project_arn)

In [ ]:
 # Upload an audio file
from IPython.display import Audio,Video, display

file_name_audio = '../../synthetic_dataset/podcastdemo.mp3'
object_name_audio = f'bda/input/{file_name_audio}'

s3_client.upload_file(file_name_audio, bucket_name_kb, object_name_audio)

file_name_video = '../../synthetic_dataset/bda.m4v'
object_name_video = f'bda/input/{file_name_video}'

s3_client.upload_file(file_name_video, bucket_name_kb, object_name_video)

In [ ]:
# Load and play an MP3 file
display(Audio(file_name_audio, autoplay=True))

In [ ]:
# Load and play an MP4 file
#Video(file_name_video,embed=True)

from IPython.display import HTML
from utils.knowledge_base_operators import play


play(file_name_video)

### BDA 작업 시작
이제 BDA API를 호출해 업로드한 오디오 파일을 처리합니다. 실습 시작 시 생성한 BDA 프로젝트 ARN을 제공하고, BDA 결과를 저장할 S3 위치를 지정해야 합니다.

BDA 비동기 작업을 호출하는 전체 API 레퍼런스는 [문서](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-data-automation-runtime/client/invoke_data_automation_async.html)를 참고하세요.

In [ ]:
# Start BDA task audio

input_name = object_name_audio
output_name = f'bda/output/' 

response_aud = bda_runtime_client.invoke_data_automation_async(
            inputConfiguration={'s3Uri':  f"s3://{bucket_name_kb}/{input_name}"},
            outputConfiguration={'s3Uri': f"s3://{bucket_name_kb}/{output_name}"},
            dataAutomationProfileArn= f'arn:aws:bedrock:us-west-2:{account_id}:data-automation-profile/us.data-automation-v1',
            dataAutomationConfiguration={
                'dataAutomationProjectArn':kb_project_arn,
                #'dataAutomationArn': kb_project_arn,
                'stage': 'DEVELOPMENT'
            })
response_aud

In [ ]:
invocation_audio_arn = response_aud.get("invocationArn")
print("BDA audio task started:", invocation_audio_arn)

In [ ]:
# Start BDA task video
input_name = object_name_video
output_name = f'bda/output/' 

response_vid = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={'s3Uri':  f"s3://{bucket_name_kb}/{object_name_video}"},
    outputConfiguration={'s3Uri': f"s3://{bucket_name_kb}/{output_name}"},
    dataAutomationProfileArn= f'arn:aws:bedrock:us-west-2:{account_id}:data-automation-profile/us.data-automation-v1',
    dataAutomationConfiguration={
        'dataAutomationProjectArn':kb_project_arn,
        'stage': 'DEVELOPMENT'
    })

response_vid

In [ ]:
invocation_video_arn = response_vid.get("invocationArn")
print("BDA video task started:", invocation_video_arn)

### 아래 코드 셀을 실행해 BDA 작업 실행 상태를 모니터링할 수 있습니다

In [ ]:
statusAudio,statusVideo, status_aud_response, status_vid_response = None, None, None, None
while (statusAudio not in ["Success","ServiceError","ClientError"]) or (statusVideo not in ["Success","ServiceError","ClientError"]):
    status_aud_response = bda_runtime_client.get_data_automation_status(
        invocationArn=invocation_audio_arn
    )
    statusAudio = status_aud_response.get("status")
    clear_output(wait=True)
   # print(f"{datetime.now().strftime('%H:%M:%S')} : BDA kb audio task: {statusAudio}")
    
    status_vid_response = bda_runtime_client.get_data_automation_status(
        invocationArn=invocation_video_arn
    )
    statusVideo = status_vid_response.get("status")
    clear_output(wait=True)
    print(f"{datetime.now().strftime('%H:%M:%S')} : "\
          f"BDA kb video task: {statusVideo} "\
          f"BDA kb audio task: {statusAudio}")
    time.sleep(5)

output_aud_config = status_aud_response.get("outputConfiguration",{}).get("s3Uri")
print("Ouput configuration file:", output_aud_config)

output_vid_config = status_vid_response.get("outputConfiguration",{}).get("s3Uri")
print("Ouput configuration file:", output_vid_config)

# 처리된 오디오 파일에 대한 BDA 출력 살펴보기

In [ ]:
out_vid_loc = status_vid_response['outputConfiguration']['s3Uri'].split("/job_metadata.json", 1)[0].split(bucket_name_kb+"/")[1]
out_vid_loc += "/0/standard_output/0/result.json"
print(out_vid_loc)
s3_client.download_file(bucket_name_kb, out_vid_loc, 'result_vid.json')

In [ ]:
data_vid = json.load(open('result_vid.json'))
print(data_vid["video"]["summary"])

In [ ]:
#download audio data file
out_aud_loc = status_aud_response['outputConfiguration']['s3Uri'].split("/job_metadata.json", 1)[0].split(bucket_name_kb+"/")[1]
out_aud_loc += "/0/standard_output/0/result.json"
print(out_aud_loc)
s3_client.download_file(bucket_name_kb, out_aud_loc, 'result_aud.json')

## 결론
축하합니다! 이 노트북을 따라 BDA를 사용해 비디오와 오디오 파일을 처리했으며, 다음 노트북인 02_audio_video_rag_kb에서 필요에 맞는 강력한 멀티모달 RAG 애플리케이션을 구축할 준비가 되었습니다.